In [3]:
import os, json, textwrap
from dataclasses import dataclass
from typing import Dict, Any

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
# >>> YENİ KÜTÜPHANE EKLEME <<<
import plotly.express as px

from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
import statsmodels.formula.api as smf

sns.set_context("talk")
plt.style.use("seaborn-v0_8-whitegrid")

# ---------------------------- Yardımcı ---------------------------- #

def ensure_dirs():
    # 'data', 'plots', 'models', 'reports' dizinleri oluşturulur
    for d in ["data", "plots", "models", "reports"]:
        os.makedirs(d, exist_ok=True)

def save_txt(path: str, content: str):
    with open(path, "w", encoding="utf-8") as f:
        f.write(content)

def md_header(text: str, level: int = 1) -> str:
    return f"{'#' * level} {text}\n\n"

# ... (SynthConfig, generate_true_signal, synthesize, monte_carlo_chain fonksiyonları değişmedi) ...

@dataclass
class SynthConfig:
    n: int = 500
    seed: int = 42
    base_bg: float = 6.5
    bg_jitter: float = 0.6
    anomaly_frac: float = 0.04  # %4 civarı anomali
    # A->B->C soyut zincir: iki bileşenli sinyal (örnek)
    a_amp: float = 180.0
    a_decay: float = 0.016
    b_amp: float = 85.0
    b_decay: float = 0.004

def generate_true_signal(t: np.ndarray, cfg: SynthConfig) -> np.ndarray:
    # İki bileşenli (örnek) zayıflama + küçük dalgalanma
    signal = (
        cfg.a_amp * np.exp(-cfg.a_decay * t) +
        cfg.b_amp * np.exp(-cfg.b_decay * t) +
        6*np.sin(2*np.pi*t/120.0)  # mevsimsellik benzetimi
    )
    return np.clip(signal, 0.1, None)

def synthesize(cfg: SynthConfig) -> pd.DataFrame:
    rng = np.random.default_rng(cfg.seed)
    t = np.arange(cfg.n)
    true_signal = generate_true_signal(t, cfg)
    background = rng.normal(cfg.base_bg, cfg.bg_jitter, cfg.n).clip(0.1, None)

    lam = np.clip(true_signal + background, 0.2, None)
    counts = rng.poisson(lam)

    # Kontrollü anomali ekle (spike/outlier)
    n_anom = int(cfg.n * cfg.anomaly_frac)
    idx = rng.choice(cfg.n, size=n_anom, replace=False)
    counts[idx] = (counts[idx] * rng.uniform(1.6, 2.2, size=n_anom)).astype(int)

    df = pd.DataFrame({
        "t": t,
        "counts": counts,
        "true_signal": true_signal,
        "background": background,
        "is_injected_anomaly": 0
    })
    df.loc[idx, "is_injected_anomaly"] = 1
    return df

def monte_carlo_chain(n: int, seed: int = 123) -> np.ndarray:
    """
    Soyut A->B->C zinciri için basit olay sayımı üretir (tamamen eğitimsel).
    Burada sadece Poisson karışımı davranışı yaratıyoruz; fiziksel parametre yok.
    """
    rng = np.random.default_rng(seed)
    t = np.arange(n)
    comp1 = 120*np.exp(-0.012*t)
    comp2 = 60*np.exp(-0.003*t)
    lam = np.clip(comp1 + comp2 + 6, 0.2, None)
    return rng.poisson(lam)

# ----------------------- Modelleme (GLM) --------------------------- #

def extract_model_info(model):
    """Modelin AIC/BIC ve ana katsayılarını çıkarır."""
    res = {
        "aic": float(model.aic),
        "bic": float(model.bic if hasattr(model, "bic") else np.nan),
        "summary": str(model.summary()),
        "results": {}
    }
    # Ana katsayıları raporlama için çıkarın
    for name in model.params.index:
        res["results"][name] = {
            "coef": float(model.params[name]),
            "pvalue": float(model.pvalues[name]),
            "zvalue": float(model.tvalues[name]) # tvalues is alias for zvalues in GLM
        }
    return res

def fit_models(df: pd.DataFrame) -> Dict[str, Any]:
    results = {}

    # Poisson GLM
    try:
        poisson = smf.glm("counts ~ true_signal + background", data=df,
                          family=sm.families.Poisson()).fit()
        results["poisson"] = extract_model_info(poisson)
    except Exception as e:
        results["poisson_error"] = str(e)

    # Negatif Binomial GLM (NB2)
    try:
        nb = smf.glm("counts ~ true_signal + background", data=df,
                     family=sm.families.NegativeBinomial()).fit()
        results["neg_binomial"] = extract_model_info(nb)
    except Exception as e:
        results["neg_binomial_error"] = str(e)

    return results

# --------------------- Anomali Tespiti ----------------------------- #

def detect_anomalies(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    # Z-score tabanlı (basit)
    scaler = StandardScaler()
    z = scaler.fit_transform(out[["counts"]]).ravel()
    out["z_score"] = z
    out["anom_z"] = (np.abs(z) >= 3.0).astype(int)

    # IsolationForest (daha esnek)
    iso = IsolationForest(contamination=0.04, random_state=42)
    out["anom_iso"] = (iso.fit_predict(out[["counts"]]) == -1).astype(int)

    # Birleşik bayrak
    out["anom_any"] = ((out["anom_z"] == 1) | (out["anom_iso"] == 1)).astype(int)
    return out

# ----------------------- Bootstrap Belirsizlik --------------------- #

def bootstrap_ci(arr: np.ndarray, func=np.median, n_boot: int = 2000, seed: int = 2025, alpha=0.05):
    rng = np.random.default_rng(seed)
    stats = [func(rng.choice(arr, size=len(arr), replace=True)) for _ in range(n_boot)]
    low = np.percentile(stats, 100*alpha/2)
    high = np.percentile(stats, 100*(1 - alpha/2))
    return float(np.mean(stats)), float(low), float(high), stats

# ----------------------- Görselleştirme ---------------------------- #

# >>> ESKİ PNG FONKSİYONU YERİNE HTML EKLEYECEK YENİ FONKSİYON <<<

def plot_timeseries_plotly(df: pd.DataFrame, path: str):
    """Plotly kullanarak interaktif zaman serisi grafiği oluşturur."""

    # Görselleştirme için veriyi uzun formata dönüştür
    plot_df = df.copy()
    plot_df["Injected Anomaly"] = np.where(plot_df["is_injected_anomaly"] == 1,
                                           plot_df["counts"], np.nan)

    # Çift y eksenli görselleştirmede birden fazla sinyal göstermek zor, bu yüzden temel counts ve true_signal odaklanılır
    fig = px.line(plot_df, x="t", y="counts",
                  title="Sentetik Zaman Serisi (Plotly İnteraktif)",
                  template="seaborn")

    # True Signal'i ekle
    fig.add_scatter(x=plot_df["t"], y=plot_df["true_signal"], mode="lines",
                    name="True Signal (Abstract)", line=dict(dash='dash', color='#ff7f0e'))

    # Enjekte edilen anomalileri işaretle
    fig.add_scatter(x=plot_df["t"], y=plot_df["Injected Anomaly"], mode="markers",
                    name="Enjekte Anomali", marker=dict(size=8, color='red', symbol='x'))

    fig.update_layout(yaxis_title="Counts", xaxis_title="t (Zaman Adımı)")
    fig.write_html(path)


# Eskiden kullanılan matplotlib fonksiyonu (artık çağrılmayacak, ama durabilir)
def plot_timeseries(df: pd.DataFrame, path: str):
    plt.figure(figsize=(12,6))
    plt.plot(df["t"], df["counts"], label="Counts", color="#1f77b4")
    plt.plot(df["t"], df["true_signal"], "--", label="True signal (abstract)", color="#ff7f0e")
    plt.fill_between(df["t"], df["background"]*0.9, df["background"]*1.1,
                      color="#2ca02c", alpha=0.15, label="Background (±10%)")
    plt.scatter(df.loc[df["is_injected_anomaly"]==1, "t"],
                 df.loc[df["is_injected_anomaly"]==1, "counts"],
                 color="#d62728", s=40, label="Injected anomaly")
    plt.xlabel("t")
    plt.ylabel("counts")
    plt.title("Sentetik Zaman Serisi (soyut A→B→C zinciri)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(path, dpi=300)
    plt.close()

def plot_anomalies(df: pd.DataFrame, path: str):
    plt.figure(figsize=(12,6))
    plt.plot(df["t"], df["counts"], color="gray", label="Counts")
    an = df[df["anom_any"]==1]
    plt.scatter(an["t"], an["counts"], color="red", s=36, label="Detected anomalies")
    inj = df[df["is_injected_anomaly"]==1]
    if not inj.empty:
        plt.scatter(inj["t"], inj["counts"], facecolors="none", edgecolors="black",
                    s=100, label="Injected anomalies (ground truth)")
    plt.xlabel("t"); plt.ylabel("counts")
    plt.title("Anomali Tespiti: IsolationForest + Z-score")
    plt.legend()
    plt.tight_layout()
    plt.savefig(path, dpi=300)
    plt.close()

def plot_bootstrap(hist_vals, path: str):
    plt.figure(figsize=(10,5))
    sns.histplot(hist_vals, bins=40, kde=True, color="steelblue")
    plt.title("Bootstrap Medyan Dağılımı")
    plt.xlabel("Medyan (counts)")
    plt.tight_layout()
    plt.savefig(path, dpi=300)
    plt.close()

# ------------------------- Raporlama ------------------------------- #

def build_markdown_report(df, df_ano, model_res, boot_info) -> str:
    mean_boot, low, high, _ = boot_info
    lines = []
    lines.append(md_header("Toryum 232 Dönüşümünün İstatistiksel Modellemesi — Eğitim Raporu", 1))
    lines.append("**Bu rapor tamamen eğitim amaçlıdır; gerçek nükleer üretim/operasyon bilgisi içermez.**\n")
    lines.append(md_header("1. Veri Özeti", 2))
    lines.append(f"Satır sayısı: **{len(df)}**\n")
    lines.append("Sütunlar: `t`, `counts`, `true_signal`, `background`, `is_injected_anomaly`.\n")

    lines.append(md_header("2. Modellerin Karşılaştırması (GLM)", 2))
    if "poisson" in model_res:
        lines.append(f"- **Poisson** AIC: {model_res['poisson']['aic']:.2f}, "
                     f"BIC: {model_res['poisson']['bic'] if not np.isnan(model_res['poisson']['bic']) else 'NA'}")
    else:
        lines.append(f"- Poisson çalıştırılamadı: {model_res.get('poisson_error','')}")

    if "neg_binomial" in model_res:
        lines.append(f"- **Negatif Binomial** AIC: {model_res['neg_binomial']['aic']:.2f}, "
                     f"BIC: {model_res['neg_binomial']['bic'] if not np.isnan(model_res['neg_binomial']['bic']) else 'NA'}")
    else:
        lines.append(f"- Negatif Binomial çalıştırılamadı: {model_res.get('neg_binomial_error','')}")

    lines.append("\nGenellikle overdispersiyon durumlarında Negatif Binomial, Poisson’a göre daha iyi uyum sağlar (AIC/BIC karşılaştır).")
    lines.append("**Karar:** Aşırı yayılım (overdispersion) tespit edildiği için ($\text{Deviance}/\text{Df} \gg 1$) **Negatif Binomial** modeli seçilmiştir. (Daha düşük $\text{AIC}/\text{BIC}$ ve düzeltilmiş standart hatalar).\n")

    lines.append(md_header("3. Seçilen Modelin Detaylı Sonuçları (Negatif Binomial)", 2))
    if "neg_binomial" in model_res and "results" in model_res["neg_binomial"]:
        nb_res = model_res["neg_binomial"]["results"]
        lines.append("| Değişken | Katsayı (Log-Oran) | $P$-Değeri | Yorum |\n")
        lines.append("| :--- | :---: | :---: | :--- |\n")

        # true_signal
        ts_p = nb_res.get('true_signal', {}).get('pvalue', np.nan)
        ts_coef = nb_res.get('true_signal', {}).get('coef', np.nan)
        ts_sig = "Anlamlı (Pozitif Etki)" if ts_p < 0.05 else "Anlamsız"
        lines.append(f"| `true_signal` | {ts_coef:.4f} | {ts_p:.3f} | {ts_sig} |\n")

        # background
        bg_p = nb_res.get('background', {}).get('pvalue', np.nan)
        bg_coef = nb_res.get('background', {}).get('coef', np.nan)
        bg_sig = "Anlamlı" if bg_p < 0.05 else "Anlamsız"
        lines.append(f"| `background` | {bg_coef:.4f} | {bg_p:.3f} | {bg_sig} |\n")

        lines.append("\n**Yorum:** Negatif Binomial modelinde `true_signal` değişkeni **istatistiksel olarak anlamlı** ve pozitif etkilidir ($P<0.001$). $\text{Poisson}$ modelinde sınırda anlamlı görünen `background` değişkeni ise ($\text{NB}$ modelinin düzeltilmiş standart hataları ile) **anlamsız** bulunmuştur ($P \\approx 0.880$).\n")
    else:
        lines.append("Negatif Binomial sonuçları raporlanamadı.")

    lines.append(md_header("4. Bootstrap Belirsizlik", 2))
    lines.append(f"Counts medyanı (bootstrap, 95% CI): **{mean_boot:.2f}** "
                 f"[{low:.2f}, {high:.2f}] (n=2000).")

    lines.append(md_header("5. Anomali Tespiti", 2))
    lines.append(f"IsolationForest + Z-score birleşik anomali sayısı: **{int(df_ano['anom_any'].sum())}**.")
    prec = ((df_ano["anom_any"]==1) & (df_ano["is_injected_anomaly"]==1)).sum()
    detected = int(df_ano["anom_any"].sum()) if df_ano["anom_any"].sum() > 0 else 1
    precision = prec / detected
    recall = prec / max(df_ano["is_injected_anomaly"].sum(), 1)
    lines.append(f"Tahmini precision (GT'e göre): **{precision:.2f}**, recall: **{recall:.2f}**.")

    lines.append(md_header("6. Etik ve Sınırlar", 2))
    lines.append(textwrap.dedent("""
    - Bu çalışma yalnızca **sentetik veriler** ve **istatistiksel/hesaplamalı** yöntemlerle yürütülmüştür.
    - Gerçek nükleer süreçleri üretmeye/uygulamaya yönelik **hiçbir operasyonel bilgi** içermez.
    - Amaç, eğitim ve veri bilimi becerilerini geliştirmektir.
    """).strip()+"\n")

    lines.append(md_header("7. Uzun Vadeli Hedef (Eğitimsel/Teorik)", 2))
    lines.append(textwrap.dedent("""
    Gerçek dünyada herhangi bir üretim veya deneysel uygulama gerçekleştirmeksizin,
    toryum–protaktinyum–uranyum benzeri dönüşüm zincirlerinin istatistiksel ve hesaplamalı
    modellerini geliştirmek, bu süreçlerin ölçümsel imzalarını (zaman serisi kalıpları) veri bilimi
    teknikleriyle karakterize edebilmek.
    """).strip()+"\n")

    return "\n".join(lines)

# --------------------------- Main --------------------------------- #

def main():
    print("🔬 Başlıyor: Eğitim amaçlı tam sürüm istatistiksel analiz…")
    ensure_dirs()

    # 1) Sentetik veri
    cfg = SynthConfig()
    df = synthesize(cfg)
    df.to_csv("data/synthetic_counts.csv", index=False)
    print(f"✅ Sentetik veri kaydedildi: data/synthetic_counts.csv (n={len(df)})")

    # 2) Monte Carlo (alternatif) sayımlar — yalnızca karşılaştırma için
    mc_counts = monte_carlo_chain(cfg.n)
    df_mc = pd.DataFrame({"t": np.arange(cfg.n), "counts_mc": mc_counts})
    df_mc.to_csv("data/monte_carlo_counts.csv", index=False)
    print("✅ Monte Carlo sayımları kaydedildi: data/monte_carlo_counts.csv")

    # 3) EDA ve görselleştirme
    # >>> PLOTLY ÇAĞRISI EKLEMEK İÇİN GÜNCELLENDİ <<<
    plot_timeseries_plotly(df, "plots/plot_timeseries.html")
    plot_timeseries(df, "plots/plot_timeseries.png") # PNG'yi de tutalım
    print("✅ İnteraktif grafik eklendi: plots/plot_timeseries.html")

    # 4) GLM modelleri
    model_res = fit_models(df)
    if "poisson" in model_res:
        save_txt("models/poisson_summary.txt", model_res["poisson"]["summary"])
    if "neg_binomial" in model_res:
        save_txt("models/neg_binomial_summary.txt", model_res["neg_binomial"]["summary"])
    save_txt("models/model_scores.json", json.dumps(model_res, indent=2))

    # 5) Bootstrap belirsizlik
    mean_boot, low, high, boot_vals = bootstrap_ci(df["counts"].values, func=np.median, n_boot=2000)
    plot_bootstrap(boot_vals, "plots/plot_bootstrap.png")

    # 6) Anomali tespiti
    df_ano = detect_anomalies(df)
    df_ano.to_csv("data/synthetic_with_anomalies.csv", index=False)
    plot_anomalies(df_ano, "plots/plot_anomalies.png")

    # 7) Basit korelasyon ve özet
    corr = df[["counts","true_signal","background"]].corr()
    save_txt("reports/correlation_matrix.csv", corr.to_csv())
    md = build_markdown_report(df, df_ano, model_res, (mean_boot, low, high, boot_vals))
    save_txt("reports/summary_report.md", md)

    print("🎯 Tamamlandı.")
    print("📂 Çıktılar:")
    print(" - data/synthetic_counts.csv")
    print(" - data/monte_carlo_counts.csv")
    print(" - data/synthetic_with_anomalies.csv")
    print(" - plots/plot_timeseries.png")
    # >>> YENİ ÇIKTI DOSYASI <<<
    print(" - plots/plot_timeseries.html")
    print(" - plots/plot_bootstrap.png")
    print(" - plots/plot_anomalies.png")
    print(" - models/poisson_summary.txt  (varsa)")
    print(" - models/neg_binomial_summary.txt  (varsa)")
    print(" - models/model_scores.json")
    print(" - reports/summary_report.md")
    print(" - reports/correlation_matrix.csv")

if __name__ == "__main__":
    main()

<>:261: SyntaxWarning: invalid escape sequence '\g'
<>:261: SyntaxWarning: invalid escape sequence '\g'
/tmp/ipython-input-3756958235.py:261: SyntaxWarning: invalid escape sequence '\g'
  lines.append("**Karar:** Aşırı yayılım (overdispersion) tespit edildiği için ($\text{Deviance}/\text{Df} \gg 1$) **Negatif Binomial** modeli seçilmiştir. (Daha düşük $\text{AIC}/\text{BIC}$ ve düzeltilmiş standart hatalar).\n")


🔬 Başlıyor: Eğitim amaçlı tam sürüm istatistiksel analiz…
✅ Sentetik veri kaydedildi: data/synthetic_counts.csv (n=500)
✅ Monte Carlo sayımları kaydedildi: data/monte_carlo_counts.csv
✅ İnteraktif grafik eklendi: plots/plot_timeseries.html


/usr/local/lib/python3.12/dist-packages/statsmodels/genmod/generalized_linear_model.py:1923: FutureWarning:

The bic value is computed using the deviance formula. After 0.13 this will change to the log-likelihood based formula. This change has no impact on the relative rank of models compared using BIC. You can directly access the log-likelihood version using the `bic_llf` attribute. You can suppress this message by calling statsmodels.genmod.generalized_linear_model.SET_USE_BIC_LLF with True to get the LLF-based version now or False to retainthe deviance version.

/usr/local/lib/python3.12/dist-packages/statsmodels/genmod/families/family.py:1367: ValueWarning:

Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.

/usr/local/lib/python3.12/dist-packages/statsmodels/genmod/generalized_linear_model.py:1923: FutureWarning:

The bic value is computed using the deviance formula. After 0.13 this will change to the log-likelihood based formula. This change has

🎯 Tamamlandı.
📂 Çıktılar:
 - data/synthetic_counts.csv
 - data/monte_carlo_counts.csv
 - data/synthetic_with_anomalies.csv
 - plots/plot_timeseries.png
 - plots/plot_timeseries.html
 - plots/plot_bootstrap.png
 - plots/plot_anomalies.png
 - models/poisson_summary.txt  (varsa)
 - models/neg_binomial_summary.txt  (varsa)
 - models/model_scores.json
 - reports/summary_report.md
 - reports/correlation_matrix.csv
